- 课后练习
- 文件IO
- 文件对象

### base64解码

In [1]:
import string

In [18]:
def b64decode(data:str) -> bytes:
    table = '{}{}{}{}'.format(string.ascii_uppercase, string.ascii_lowercase,string.digits, '+/')
    print(table)
    decoded = bytearray()
    s = 0
    for e in range(4, len(data) + 1, 4):
        # tmp 为临时变量，记录位移
        print(e)
        tmp = 0
        for i, c in enumerate(data[s:e]):
            print('{} => {}'.format(i, c))
            if c != '=':
                print(tmp)
                print(c)
                tmp += table.index(c) << 24 - (i + 1) * 6
            else:
                tmp += 0 << 24 - (i + 1) * 6
        decoded.extend(tmp.to_bytes(3, 'big'))
        s += 4
        
    return bytes(decoded.rstrip(b'\x00'))

In [21]:
b64decode('YWJjZA==')

ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/
4
0 => Y
0
Y
1 => W
6291456
W
2 => J
6381568
J
3 => j
6382144
j
8
0 => Z
0
Z
1 => A
6553600
A
2 => =
3 => =


b'abcd'

In [20]:
data

NameError: name 'data' is not defined

### 带过期的LRU缓存装饰器

- 缓存的效率（缓存对象多到一亿条的时候）

- LRU, least recently used,
- LFU, least frequently used 
- FIFO, 先进先出


可以把最后一次使用的时间戳  扔到value中

In [22]:
import datetime
import inspect
import functools

In [48]:
def cache(maxsize=128, expire=0):
    def make_key(fn, args, kwargs):
        key = []

        names = set()

        params = inspect.signature(fn).parameters
        for i, arg in enumerate(args):
            name = list(params.keys())[i]
            key.append((name, arg))

            names.add(name)

        key.extend(kwargs.items())
        names.update(kwargs.keys())

        for k, v in params.items():
            if k not in names:
                key.append((k, v.default))

        key.sort(key=lambda x: x[0])

        key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])
        return key
    
    
    def _cache(fn):
        data = {}
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = make_key(fn, args, kwargs)
            now = datetime.datetime.now().timestamp()
            
            # 命中缓存
            if key in data.keys():
                value, timestamp, _ = data[key]
                if expire == 0 or now - timestamp < expire:
                    data[key] = (value, timestamp, now)
                    print('hint, {}'.format(data))
                    return value
                else:
                    data.pop(key)
            
            # 未命中的情况
            value = fn(*args, **kwargs)
            # 需要替换
            # 把过期的值 给 换出
            if len(data) >= maxsize:
                if expire != 0:
                    expires = set()
                    for k, (_, timestamp, _) in data.items():
                        if now - timestamp >= expire:
                            expires.add(k)
                    for k in expires:
                        data.pop(k)
                
            if len(data) >= maxsize:
                k = sorted(data.items(), key=lambda x: x[1][2])[0]
                data.pop(k)
                        
            data[key] = (value, now, now)
            print('missed, {}'.format(data))
            return value
        return wrap
    return _cache

In [49]:
@cache(maxsize=2, expire=5)
def add(x, y):
    return x + y

In [60]:
add(11, 2)

missed, {'x=15&y=2': (17, 1507342143.714536, 1507342146.067659), 'x=11&y=2': (13, 1507342454.907118, 1507342454.907118)}


13

In [39]:
{'x=1&y=2': (3, 123, 234)}

{'x=1&y=2': (3, 123, 234)}

In [42]:
for k in {'x=1&y=2': (3, 123, 234), 'b': 2}.items():
    print(k)

('x=1&y=2', (3, 123, 234))
('b', 2)


In [61]:
def cache(maxsize=128, expire=0):
    def make_key(fn, args, kwargs):
        key = []

        names = set()

        params = inspect.signature(fn).parameters
        for i, arg in enumerate(args):
            name = list(params.keys())[i]
            key.append((name, arg))

            names.add(name)

        key.extend(kwargs.items())
        names.update(kwargs.keys())

        for k, v in params.items():
            if k not in names:
                key.append((k, v.default))

        key.sort(key=lambda x: x[0])

        key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])
        return key
    
    
    def _cache(fn):
        data = {}
        
        queue = []
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = make_key(fn, args, kwargs)
            now = datetime.datetime.now().timestamp()
            
            # 命中缓存
            if key in data.keys():
                value, timestamp, _ = data[key]
                
                queue.remove(key)
                
                if expire == 0 or now - timestamp < expire:
                    # data[key] = (value, timestamp, now)
                    queue.insert(0, key)
                    print('hint, {}'.format(data))
                    return value
                else:
                    data.pop(key)
            
            # 未命中的情况
            value = fn(*args, **kwargs)
            # 需要替换
            # 把过期的值 给 换出
            if len(data) >= maxsize:
                if expire != 0:
                    expires = set()
                    for k, (_, timestamp, _) in data.items():
                        if now - timestamp >= expire:
                            expires.add(k)
                    for k in expires:
                        queue.remove(k)
                        
                        data.pop(k)
                
            if len(data) >= maxsize:
                
                k = queue.pop()
                # k = sorted(data.items(), key=lambda x: x[1][2])[0]
                data.pop(k)
                        
            data[key] = (value, now, now)
            
            queue.insert(0, key)
            print('missed, {}'.format(data))
            return value
        return wrap
    return _cache

In [62]:
@cache(maxsize=2, expire=5)
def add(x, y):
    return x + y

In [66]:
add(1, 3)

missed, {'x=3&y=5': (8, 1507344291.040315, 1507344291.040315), 'x=1&y=3': (4, 1507344295.376191, 1507344295.376191)}


4

In [64]:
add(2, 4)

missed, {'x=1&y=3': (4, 1507344284.793847, 1507344284.793847), 'x=2&y=4': (6, 1507344287.559317, 1507344287.559317)}


6

In [78]:
add(3, 5)

missed, {'x=1&y=3': (4, 1507344295.376191, 1507344295.376191), 'x=3&y=5': (8, 1507344322.579435, 1507344322.579435)}


8

In [79]:
from collections import namedtuple

In [80]:
Node = namedtuple('Node', ['data', 'prev', 'next'])
Item = namedtuple('Item', ['key', 'value', 'timestamp'])

In [81]:
def linked_list():
    _head = None
    _tail = None
    
    # put pop remove
    
    def put(item):
        nonlocal _head
        nonlocal _tail
        if _head is None:
            _head = [item, None, None]
        else:
            node = [item, None, _head]
            _head[1] = node
            _head = node
        
        if _tail is None:
            _tail = _head
        return _head
    
    def pop():
        nonlocal _tail
        if _tail is None:
            _head = None
            return None
        
        node = _tail
        _tail = node[1]
        return node
    
    def remove(node):
        nonlocal _head
        nonlocal _tail
        if node is _head:
            _head = node[2]
        if node is _tail:
            pop()
            return
        
        node[1][2] = node[2]
        node[2][1] = node[1]
        
    return put, pop, remove

In [82]:
put, pop, remove = linked_list()

In [83]:
put(1)

[1, None, None]

In [84]:
put(2)

[2, None, [1, [...], None]]

In [85]:
put(3)

[3, None, [2, [...], [1, [...], None]]]

In [86]:
node = put(4)

In [87]:
put(5)

[5, None, [4, [...], [3, [...], [2, [...], [1, [...], None]]]]]

In [88]:
remove(node)

In [89]:
pop()

[1, [2, [3, [5, None, [...]], [...]], [...]], None]

In [90]:
def cache(maxsize=128, expire=0):
    def make_key(fn, args, kwargs):
        key = []

        names = set()

        params = inspect.signature(fn).parameters
        for i, arg in enumerate(args):
            name = list(params.keys())[i]
            key.append((name, arg))

            names.add(name)

        key.extend(kwargs.items())
        names.update(kwargs.keys())

        for k, v in params.items():
            if k not in names:
                key.append((k, v.default))

        key.sort(key=lambda x: x[0])

        key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])
        return key
    
    
    def _cache(fn):
        data = {}
        
        queue = []
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = make_key(fn, args, kwargs)
            now = datetime.datetime.now().timestamp()
            
            # 命中缓存
            if key in data.keys():
                # value, timestamp, _ = data[key]
                node = data[key]
                # queue.remove(key)
                item = node[0]
                
                reove(node)
                
                
                if expire == 0 or now - timestamp < expire:
                    # data[key] = (value, timestamp, now)
                    # queue.insert(0, key)
                    data[key] = put(item)
                    print('hint, {}'.format(data))
                    return value
                else:
                    data.pop(key)
            
            # 未命中的情况
            value = fn(*args, **kwargs)
            # 需要替换
            # 把过期的值 给 换出
            if len(data) >= maxsize:
                if expire != 0:
                    expires = set()
                    
                    for k, node in data.items():
                        if now - node[0].timestamp >= expire:
                            pop(node)
                            expires.add(node)
                    for k in expires:
                    
#                     for k, (_, timestamp, _) in data.items():
#                         if now - timestamp >= expire:
#                             expires.add(k)
#                     for k in expires:
#                         queue.remove(k)
                        
                        data.pop(k)
                
            if len(data) >= maxsize:
                
#                 k = queue.pop()
                # k = sorted(data.items(), key=lambda x: x[1][2])[0]
                
                node = pop()
            
                data.pop(node[0].key)
                        
#             data[key] = (value, now, now)
            
            node = put(Item(key, value, now))
    
#             queue.insert(0, key)
            print('missed, {}'.format(data))
            return value
        return wrap
    return _cache

In [91]:
def linked_list():
    _head = None
    _tail = None
    
    # put pop remove
    
    def put(item):
        nonlocal _head
        nonlocal _tail
        if _head is None:
            _head = {'data': item, 'prev': None. 'next': None}
        else:
            # node = [item, None, _head]
            node = {'data': item, 'prev': None. 'next': _head}
            
            # _head[1] = node
            _head['prev'] = node
            _head = node
        
        if _tail is None:
            _tail = _head
        return _head
    
    def pop():
        nonlocal _tail
        if _tail is None:
            _head = None
            return None
        
        node = _tail
        _tail = node[1]
        return node
    
    def remove(node):
        nonlocal _head
        nonlocal _tail
        if node is _head:
            _head = node[2]
        if node is _tail:
            pop()
            return
        
        node[1][2] = node[2]
        node[2][1] = node[1]
        
    return put, pop, remove

### 命令分发器

In [92]:
def commands():
    commands = {}
    
    
    def register(command):
        def _register(fn):
            commands[command] = fn
            return fn
        return _register
        
    def default_fn():
        print('undefined command')
        
    def run():
        while True:
            cmd = input('>>')
            if cmd.strip() == 'quit':
                return

            commands.get(cmd.strip(), default_fn)()
            
    return register, run

In [93]:
register, run = commands()

In [94]:
run()

>>sadfsdf
undefined command
>>quit


In [99]:
@register('add')
def add():
    print(1)

In [105]:
def dispatcher(default_handler=None):
    commands = {}
    
    if default_handler is None:
        default_handler = lambda *args, **kwargs: print('not found')
    
    def register(command):
        def _register(fn):
            commands[command] = fn
            return fn
        return _register
        
        
    def run():
        while True:
            command, _, params = input('>>').partition(':')
            if command.strip() == 'quit':
                return
            args = []
            kwargs = {}
            
            for param in params.split(','):
                if '=' in param:
                    kwargs.update([param.split('=')])
                else:
                    args.append(param)
            
            ret = commands.get(command, default_handler)(*args, **kwargs)
            
            if ret is not None:
                print(ret)
            
    return register, run

In [106]:
register, run = dispatcher()

In [107]:
@register('add')
def add(x, y):
    return x + y

In [143]:
import inspect
from collections import namedtuple

def dispatcher(default_handler=None):
    Handler = namedtuple('Handler', ['fn', 'params'])
    commands = {}
    
    if default_handler is None:
        default_handler = lambda *args, **kwargs: print('not found')
    
    def register(command):
        def _register(fn):
            # commands[command] = fn
            params = inspect.signature(fn).parameters
            commands[command] = Handler(fn, params)
            return fn
        return _register
        
        
    def run():
        while True:
            command, _, params = input('>>').partition(':')
            if command.strip() == 'quit':
                return
            
            
            handler = commands.get(command)
            
            args = []
            kwargs = {}
            


            for i, param in enumerate(params.split(',')):
                print(param)
                if '=' in param:
                    name, _, vale = param.partition('=')
                    anno = handler.params[name.strip()].annotation
                    
                    if anno != inspect.Parameter.empty:
                        kwargs[name.strip()] = anno(value)
                    else:
                        kwargs[name.strip()] = value
                else:
                    print(list(handler.params.values())[i])
                    anno = list(handler.params.values())[i].annotation
                    print(anno, param)
                    if anno != inspect.Parameter.empty:
                         args.append(anno(param.strip()))
                    else:
                        args.append(param.strip())
                
            
            # ret = commands.get(command, default_handler)(*args, **kwargs)
            ret = handler.fn(*args, **kwargs)
            if ret is not None:
                   
                
                print(ret)
            
    return register, run

In [144]:
register, run = dispatcher()

In [145]:
@register('add')
def add(x:int, y:int):
    return x + y

In [146]:
run()

>>add:1,2
1
x:int
<class 'int'> 1
2
y:int
<class 'int'> 2
3
>>quit


In [147]:
run()

>>add:

x:int
<class 'int'> 


ValueError: invalid literal for int() with base 10: ''

In [148]:
import inspect
from collections import namedtuple

def dispatcher(default_handler=None):
    Handler = namedtuple('Handler', ['fn', 'params'])
    commands = {}
    
    if default_handler is None:
        default_handler = lambda *args, **kwargs: print('not found')
    
    def register(command):
        def _register(fn):
            # commands[command] = fn
            params = inspect.signature(fn).parameters
            commands[command] = Handler(fn, params)
            return fn
        return _register
        
        
    def run():
        while True:
            command, _, params = input('>>').partition(':')
            if command.strip() == 'quit':
                return
            
            
            handler = commands.get(command, Handler(default_handler, {}))
            
            args = []
            kwargs = {}
            
            param_values = list(handler.params.values())

            for i, param in enumerate(params.split(',')):
                print(param)
                if '=' in param:
                    name, _, vale = param.partition('=')
#                     anno = handler.params[name.strip()].annotation
                    
#                     if anno != inspect.Parameter.empty:
#                         kwargs[name.strip()] = anno(value)
                        
                    p = handler.params.get(name.strip())
                    if p is not None and p.annotation != inspect.Parameter.empty:
                        kwargs[name.strip()] = p.annotation(value)
                        
                    else:
                        kwargs[name.strip()] = value
                else:
                    
                    
#                     anno = list(handler.params.values())[i].annotation
#                     if anno != inspect.Parameter.empty:
#                          args.append(anno(param.strip()))
                    if len(param_values) > i and param_values[i].annotation != inspect.Parameter.empty:
                        args.append(param_values[i].annotation(param.strip()))
                    else:
                        args.append(param.strip())
                
            
            # ret = commands.get(command, default_handler)(*args, **kwargs)
            ret = handler.fn(*args, **kwargs)
            if ret is not None:
                   
                
                print(ret)
            
    return register, run

In [149]:
register, run = dispatcher()

In [150]:
@register('add')
def add(x:int, y:int):
    return x + y

In [152]:
run()

>>add:1, 
1
 


ValueError: invalid literal for int() with base 10: ''

### 优先队列

In [154]:
def queue():
    lst = []
    
    # p 是 优先级
    def put(value, p):
        for i, item in enumerate(lst):
            if item[1] <= p:
                lst.insert(i, (value, p))
                break
        else:
            lst.append((value, p))
            
    def pop():
        if lst:
            return lst.pop()[0]
        else:
            return None
        
    return put, pop

In [155]:
put, pop = queue()

In [156]:
put(3, 5)

In [157]:
put(2, 3)

In [158]:
pop()

2

In [159]:
put(1, 1)

In [160]:
pop()

1

- 完全二叉树
- 父节点总是比子节点要大
- 完全二叉树可以使用列表来表示
- data[0] 是根节点
- data[i].left = data[2i+1]
- data[i].right = data[2i+2]
- data[i].parent = data[floor((i-2)/2)

In [161]:
def heap():
    data = []
    
    # p 是 优先级
    def put(value, p):
        
        i = len(data)
        data.append((value, p))
        
        pi = (i-1) // 2
        
        while pi >= 0 and p > data[pi][1]:
            data[i], data[pi] = data[pi], data[i]
            
            
            i = pi
            pi = (i-1) // 2
        
            
    def pop():
        if not data:
            return None
        if len(data) == 1:
            return data[0]
        
        i = 0
        ret = data[i]
        data[i] = data.pop()
        
        li = 2 * i + 1
        ri = 2 * i + 2
        
        while data and li < len(data):
            ci = li
            if ri < len(data) and data[ri][1] > data[li][1]:
                ci = ri
            if data[ci][1] > data[i][1]:
                data[ci], data[i] = data[i], data[ci]
                i = ci
                li = 2 * i + 1
                ri = 2 * i + 2
            else:
                break
        return ret
    return put, pop

In [162]:
put, pop = heap()

In [163]:
put(0, 35)

In [164]:
put(0, 28)

In [165]:
put(0, 28)

In [166]:
put(0, 78)

In [172]:
pop()

(0, 28)

In [173]:
import random


random.randint(0, 100)

99

In [174]:
lst = [random.randint(0, 100) for x in range(20)]

In [175]:
for x in lst:
    put(0, x)

In [176]:
for _ in lst:
    print(pop()[1])

98
91
90
75
73
73
72
63
59
44
29
28
27
26
24
23
21
13
12
10
